<a href="https://colab.research.google.com/github/juliowaissman/transformadores/blob/main/transformadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://huggingface.co/front/assets/huggingface_logo.svg)

# Transformadores y modelos modernos de PLN

## Curso de Procesamiento de Lenguaje Natural

### Maestría en Ciencia de Datos

### Universidad de Sonora

**Olivia Gutú** y **Julio Waissman** 

Los *transformadores* son modelos de redes neuronales que están diseñados para manejar datos secuenciales, como el lenguaje natural. A diferencia de los RNN, los transformadores utilizan un modelo que se conoce como *mecanismo de atención* que no requiere que los datos secuenciales se procesen en orden. 

Por ejemplo, si los datos de entrada son una oración en lenguaje natural, el transformador no necesita procesar el principio antes del final. Debido a esta característica, el transformador permite mucha más paralelización que los RNN y, por lo tanto, reduce los tiempos de entrenamiento.

Estos modelos se están utilizando muy intensivamente para abordar muchos problemas de PNL, reemplazando los modelos de redes neuronales recurrentes más antiguos, como los LSTM. Dado que estos modelos facilitan una mayor paralelización durante el entrenamiento, ha permitido el entrenamiento en conjuntos de datos más grandes de lo que era posible antes de su introducción.

Una buena explicación de lo que son los transformadores, y la idea subyaciente del mecanismo de atención se puede consultar en [este artículo de Medium](https://towardsdatascience.com/transformers-141e32e69591).

En esta libreta vamos a explorar el uso de la librería `transformers` de la empresa [Hugging Face](https://huggingface.co). Esta librería es un *hub* que permite cargar y aplicar diferentes modelos preentrenados de transformadores para diferentes tareas. Igualmente es posible realizar un entrenamiento fino de los modelos a casos particulares, y someter dichos modelos para que puedan ser utilizados por otros.

Para instalar `transformers` es necesario contar con *TensorFlow 2.x* y/o *pyTorch*. En general la mayoría de los modelos vienen en *pyTorch*. Si lo ejecutas desde *Colab*, el entorno ya viene con ambas librerias tensoriales preinstaladas.

Par algunos modelos es necesario contar con [el módulo `sentencepiece`](https://github.com/google/sentencepiece) que es un módulo desarrollado por google para tokenizar, basado solo en la estructura de unigramas (independiente del lenguaje). Viene con funciones para acelerar tokenizadores existentes.

In [ ]:
!pip install transformers
!pip install sentencepiece

La librería viene con un comando llamado `pipeline` que permite seleccionar un modelo preentrenado y aplicarlo a una serie de tareas (si el modelo viene adaptado a la tarea). La interfase es realmente sencilla de utilizar y si tenemos un modelo preentrenado que sea suficiente es una manera fácil de utilizar modelos modernos en aplicaciones.

Las tareas para las que se puede utilizar el comando `pipeline`son:

- `"feature-extraction"`
- `"sentiment-analysis"`
- `"ner"`
- `"question-answering"`
- `"fill-mask"`
- `"summarization"`
- `"translation_xx_to_yy"`
- `"text2text-generation"`
- `"text-generation"`
- `"zero-shot-classification:`
- `"conversational"`

Vamos a explorar la aplicación de alguna de ellas en esta libreta. Para mayor información del uso de `pipeline`puedes consultar [la documentación correspondiente](https://huggingface.co/transformers/task_summary.html).

In [2]:
from transformers import pipeline

# Si quieres ver la documentación de pipeline solo descomenta esta linea
#help(pipeline)

Para usar el `pipeline` es necesario seleccionar un modelo preentrenado. En la documentación se muestra [una serie de modelos generales y sus características](https://huggingface.co/transformers/model_summary.html). Lo mejor es ir y consultar la [lista exhaustiva de modelos preentrenados](https://huggingface.co/models) de *Hugging Face*.

Vamos a practicar con algunas tareas.

## Contestando preguntas

Para ejemplificar esta tarea vamos a descargar el texto de una conferencia Matutina del Presidente de México del [proyecto público de NOSTRODATA](https://github.com/NOSTRODATA/conferencias_matutinas_amlo), y nos vamos a quedar con el discurso inicial.

In [ ]:
import pandas as pd
import pprint

file = 'https://raw.githubusercontent.com/NOSTRODATA/conferencias_matutinas_amlo/master/2021/4-2021/abril%2014%2C%202021/mananera_14_04_2021.csv'
df = pd.read_csv(file)
    
ind = min(df.index[df.Participante != df.Participante[0]]) - 1
contexto = "\n".join(df.Texto[:ind])

pprint.pprint(contexto)

Ahora vamos autilizar un modelo tipo [DistilBERT](https://arxiv.org/pdf/1910.01108.pdf) entrenado en español para responder preguntas. 

In [ ]:
nlp = pipeline(
    'question-answering', 
    model='mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
    tokenizer=(
        'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',  
        {"use_fast": False}
    )
)

preguntas = [
  '¿Qué compañia se constituyó?',
  '¿Cual es la audiencia potencial?',
  '¿Que les pidió?'
] 

for pregunta in preguntas:
    respuesta = nlp({'question': pregunta, 'context': contexto})
    print("=" * 30)
    print(f"Pregunta: {pregunta}")
    print(f"Respuesta: \"{respuesta['answer']}\", con un score de {respuesta['score']}")
print("=" * 30) 

**Prueba ahora con otro texto (otra mañanera, o una entrada de Wikipedia por ejemplo) y revis el resultado.**

**¿Es el único modelo para responer preguntas en un contexto en español entre los modelos de Hugging Faces? Si hay otro(s) anotalos aqui mismo.**

## Haciendo resumenes de un texto

¿Y como podríamos resumir el documento que acabamos de ver? Vamos a probar un modelo llamad [mT5](https://arxiv.org/pdf/2010.11934.pdf) para hacer resumenes.

In [ ]:
resumidor = pipeline(
    "summarization", 
    model="LeoCordoba/mt5-small-mlsum"
)
resumen = resumidor(contexto, min_length=5, max_length=500)
print("\n\nResumen:\n\n" + resumen[0]['summary_text'])

**Prueba cambiando los parámetros que puedes modificar (`min_lenght`y `max_lenght`). Revisa el resumen de otro texto (como una descripción de un sitio turístico por ejemplo).**

**¿Qué otros modelos existen para hacer resúmenes de textos en español?**

## Clasificación *zero-shot*

La clasificación *zero shot* es un método semi-supervisado, en el cual a un documento se le da una serie de opciones, para la cual no estuvo previamente entrenado el sistema, pero en función del contexto (vectores de palabras de nuevo) el modelo trata de inferir las categorias más probables.

In [ ]:
classificador = pipeline(
    "zero-shot-classification", 
    model="Recognai/bert-base-spanish-wwm-cased-xnli"
)

textos = [
    "El autor se perfila, a los 50 años de su muerte, como uno de los grandes de su siglo",
    "Se realizó la fusión de Televisa con Univisión y se constituyó la compañía de medios en español más grande del mundo.",
    "El Real Madrid de nuevo se encuentra disputando la semifinal de la Copa de Campeones de la UEFA",
    "La proxima semana inicia la vacunación contra COVID para el personal de educación",
    "Aglomeraciones y falta de medidas se presentan en los mítines y eventos de las campañas de todos los candidatos"
]
etiquetas = ["cultura", "sociedad", "economia", "politica", "salud", "deportes"]

for texto in textos:
    resultado = classificador(
        texto, 
        candidate_labels=etiquetas, 
        hypothesis_template="Este texto es sobre {}."
    )
    print("=" * 80)
    print(result['sequence'])
    for label, score in zip(result['labels'], result['scores']):
        print(f"\t\t{label} ({score})")
print("=" * 80)


**Prueba cambiando categorías, el template de la hipótesis y los textos, para entender como funciona el método**

**¿exiten otros modelos preentrenados para hacer clasificación *zero-shot*? Si es el caso, enuncia uno.**

## RuPERTa 

Hasta ahorita pareciera cosa mágica, pero es importante estar pendientes que no todos los modelos basados en transformadores son buenos. Algunos se han entrenado con realmente muy pocos casos y solo funcionan correctamente en casos muy estandar. 

Para ilustrar eso, así de como se toma un modelo que sirve para varios propósitos, pero con malos resultados, vamos autilizar el modelo *RuPERTa* que es el modelo [RoBERTa](https://arxiv.org/pdf/1907.11692.pdf) entrenado en un corpus grande en español.

El modelo es muy útil y se aplica en diferentes tareas con éxito, pero en algunas apñicaciones su desempeño deja mucho que desear (sobre todo si no se utiliza un manejo del lenguaje similar al que se hace en España). Tenemos que entrenar un modelo *mexicano*.

Empecemos por hacer POS-Tagging

In [ ]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

pos2label = {
    "0": "O",
    "1": "ADJ",
    "2": "ADP",
    "3": "ADV",
    "4": "AUX",
    "5": "CCONJ",
    "6": "DET",
    "7": "INTJ",
    "8": "NOUN",
    "9": "NUM",
    "10": "PART",
    "11": "PRON",
    "12": "PROPN",
    "13": "PUNCT",
    "14": "SCONJ",
    "15": "SYM",
    "16": "VERB"
}


tokenizer = AutoTokenizer.from_pretrained('mrm8488/RuPERTa-base-finetuned-pos')
model = AutoModelForTokenClassification.from_pretrained('mrm8488/RuPERTa-base-finetuned-pos')

text = (
    "Julio Waissman, profesor de la Universidad de Sonora, " +
    "nació en Cuauhtemoc, Chihuahua " +
    "y está pensando en viajar a Puebla en vacaciones."
)

input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)
outputs = model(input_ids)
last_hidden_states = outputs[0]

for m in last_hidden_states:
  for index, n in enumerate(m):
    if(index > 0 and index <= len(text.split(" "))):
      print(text.split(" ")[index-1] + ": " + pos2label[str(torch.argmax(n).item())])


Bastante, bastante mal ¿verdad? Ahora veamos como funciona el NER:

In [ ]:
ner2label = {
    "0": "B-LOC",
    "1": "B-MISC",
    "2": "B-ORG",
    "3": "B-PER",
    "4": "I-LOC",
    "5": "I-MISC",
    "6": "I-ORG",
    "7": "I-PER",
    "8": "O"
}
tokenizer = AutoTokenizer.from_pretrained('mrm8488/RuPERTa-base-finetuned-ner')
model = AutoModelForTokenClassification.from_pretrained('mrm8488/RuPERTa-base-finetuned-ner')

input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)
outputs = model(input_ids)
last_hidden_states = outputs[0]

for m in last_hidden_states:
  for index, n in enumerate(m):
    if(index > 0 and index <= len(text.split(" "))):
      print(text.split(" ")[index-1] + ": " + ner2label[str(torch.argmax(n).item())])

Y para terminar de desencantarnos de RuPERTa vamos a terminar usando el modelo para llenar espacios vacios:


In [ ]:
pipeline_fill_mask = pipeline(
    "fill-mask", 
    model='mrm8488/RuPERTa-base'
)

rellena = pipeline_fill_mask("México es un país muy <mask> en el mundo")

print("\n\n\n\nMéxico es un país muy <mask> en Latinoamérica\n")
print("=" * 30)
for caso in rellena:
    print(f"{caso['sequence']} ({caso['score']})")
print("=" * 30)


**¿No hay otros modelos para llenar espacio vacío en español con resultados decentes? Revisa si hay otro y aplicalo a ver si da mejores resultados.**

**¿Porqué haríamos el NER o el POS tagging con transformadores, teniendo el modelo preentrenado de SpaCy?**

## Generación de texto

La generación de texto tuvo un [hito importante con el modelo GPT-2](https://openai.com/blog/better-language-models/), tanto que el modelo *GPT-3* lo mantuvieron privado por la capacidad que tenía para generar texto falso creible, y su potencial aplicación en el desarrollo de bots de noticias falsas.

Para terminar con los transformadores, vamos a ver un modelo sencillo en español para generación de texto.

In [ ]:
generador = pipeline(
    "text-generation",
    model="datificate/gpt2-small-spanish"
)

texto_generado = generador(
    "Se encontraron sapos alucinógenos en la Isla Tiburon", 
    max_length=100,
)
for muestra in texto_generado:
  pprint.pprint(muestra['generated_text'])

In [ ]:
texto_generado = generador(
    "¿Quén es Pito Pérez?", 
    max_length=100,
)
for muestra in texto_generado:
  pprint.pprint(muestra['generated_text'])

**¿Hay otros modelos mejor entrenados en español para la generación de texto automático? Revisa diferentes modelos**

**Con el modelo encontrado ¿Se podrían generar bots maliciosos para esparcir *fakes news* en tuiter?**